In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support

In [2]:
df = pd.read_csv('train.csv')

## Reading Data

### first 10 and bottom 10 rows

In [3]:
#1
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
df.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


### Storing features in 'X' and Output Variable (Survived) in 'Y'

In [5]:
#2
X = df.iloc[:,2:]
print("Checking Null Values for X")
X.isnull().sum()

Checking Null Values for X


Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
#3 a
#Cabin has many Null values. Hence, we drop it
X.drop('Cabin', axis=1, inplace=True)

In [7]:
Y = df['Survived']
Y = pd.DataFrame(Y)
#Y = Y.apply(pd.to_numeric)
print("Checking Null Values for Y")
Y.isnull().sum()


Checking Null Values for Y


Survived    0
dtype: int64

In [8]:
#Calculating mean age of Female and Male to fill NaN values
age = X.groupby('Sex', as_index=False).Age.mean()
mean_female_age = int(age['Age'][0])
mean_male_age = int(age['Age'][1])

In [9]:
#Replacing NaN values in Age column with respect to Gender
for i in range(len(X['Age'])):
    if math.isnan(X['Age'][i]):
        if X['Sex'][i] == 'male':
            X['Age'][i] = mean_male_age
        else:
            X['Age'][i] = mean_female_age


C:\Users\Divyansh K. Pathak\AppData\Local\Temp\ipykernel_20320\3941879185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'][i] = mean_male_age
C:\Users\Divyansh K. Pathak\AppData\Local\Temp\ipykernel_20320\3941879185.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'][i] = mean_female_age


In [10]:
#X.dropna(inplace=True)

In [11]:
#All NaN values are handled
X.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    2
dtype: int64

## Data Pre-processing

### Transforming categorical features into numerical features

In [12]:
#3 b
df_categorical = X.select_dtypes(include='object')

  

label_encoder = preprocessing.LabelEncoder()

for i in df_categorical.columns:
    X[i]= label_encoder.fit_transform(X[i])


In [13]:
#3 c
X=MinMaxScaler().fit_transform(X)
X

array([[1.        , 0.12134831, 1.        , ..., 0.76911765, 0.01415106,
        0.66666667],
       [0.        , 0.21348315, 0.        , ..., 0.87647059, 0.13913574,
        0.        ],
       [1.        , 0.39662921, 0.        , ..., 0.98382353, 0.01546857,
        0.66666667],
       ...,
       [1.        , 0.46404494, 0.        , ..., 0.99264706, 0.04577135,
        0.66666667],
       [0.        , 0.09101124, 1.        , ..., 0.01176471, 0.0585561 ,
        0.        ],
       [1.        , 0.24719101, 1.        , ..., 0.68529412, 0.01512699,
        0.33333333]])

### Training Logistic Regression Model

In [14]:
#4
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size= 0.2)
model= LogisticRegression()
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)

C:\Users\Divyansh K. Pathak\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
accuracy = accuracy_score(Y_test,Y_pred)
accuracy

0.8324022346368715

In [16]:
confusion = confusion_matrix(Y_test, Y_pred)
confusion

array([[91, 11],
       [19, 58]], dtype=int64)

In [17]:
fscore = precision_recall_fscore_support(Y_test,Y_pred,average='macro')
fscore

(0.8339262187088274, 0.8227018079959256, 0.8265055569914707, None)